<a href="https://colab.research.google.com/github/fhfmendes04/face_recognition_4models/blob/main/MLP_Classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import cv2
from PIL import Image
from skimage import data, exposure
from skimage.feature import hog,local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from datetime import datetime


from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

#from Models import ANN, Multinomial_LogReg

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

In [2]:
from sklearn.datasets import fetch_olivetti_faces
dataset = fetch_olivetti_faces(random_state=42)

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


In [3]:
X = dataset.images
y = dataset.target

In [4]:
from collections import Counter 
d = Counter(y)
print("Quantidade de pessoas:",len(d))
print(d)

Quantidade de pessoas: 40
Counter({0: 10, 1: 10, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10, 7: 10, 8: 10, 9: 10, 10: 10, 11: 10, 12: 10, 13: 10, 14: 10, 15: 10, 16: 10, 17: 10, 18: 10, 19: 10, 20: 10, 21: 10, 22: 10, 23: 10, 24: 10, 25: 10, 26: 10, 27: 10, 28: 10, 29: 10, 30: 10, 31: 10, 32: 10, 33: 10, 34: 10, 35: 10, 36: 10, 37: 10, 38: 10, 39: 10})


# Entrada: vetor de imagem e saída(one-hot) para MLP.

In [5]:
def lbp_features(X): 
    METHOD = 'uniform'
    radius = 16
    n_points = 4
    fd_list = []
    for row in X:
        lbp_image = local_binary_pattern(row, n_points, radius, METHOD)
        fd_list.append(lbp_image.flatten())
    
    return np.array(fd_list)

In [6]:
image_lbp= lbp_features(X)
print(image_lbp.shape)

(400, 4096)


In [ ]:
def hog_features(X): #====>>>>>>Usei a segunda saída da funcção que é uma imagem.
    fd_list = []
    for row in X:
        _, hog_image = hog(row, orientations=6, pixels_per_cell=(4, 4), cells_per_block=(4, 4), visualize=True)
        fd_list.append(hog_image.flatten())
    
    return np.array(fd_list)

In [ ]:
image_hog = hog_features(X)
print(image_hog.shape)

(400, 4096)


In [ ]:
print("HOG:",image_hog.max(), image_hog.min())
print("LBP:", image_lbp.max(), image_lbp.min())

In [7]:
# transformar categorias em one-hot-encoding: Saída
saida = y.max()+1
print("Quantidade de pessoas:",saida)
target = keras.utils.to_categorical(y, saida)

Quantidade de pessoas: 40


# MLP


In [9]:
from sklearn.model_selection import KFold
from numpy.random import seed
import plotly.express as px
from sklearn.metrics import accuracy_score

In [10]:
def bissecao_rna(dEdA, dEdB, A, B, X, Yd, N):
    
    def h_linha(au, dEdA, dEdB, dv, X, Yd, N):
      An = A - au*dEdA;
      Bn = B - au*dEdB;

      dJdAn, dJdBn = calculo_gradiente(X, Yd, An, Bn, N)
      g=np.array((dJdAn.flatten(),dJdBn.flatten()))
      
      return  (np.sum(dv[0] * g[0]) + np.sum(dv[1] * g[1]))
    
    dv = np.array((-dEdA.flatten(),-dEdB.flatten())) 
    
    au = np.random.rand() # Chute inicial

    al = 0.0
    au = np.random.rand() # Chute inicial
    it = 0

    hl = h_linha(au, dEdA, dEdB, dv, X, Yd, N)

    while hl < 0: #Verifica au está acima de zero.
      al = au
      au = au * 2
      hl = h_linha(au, dEdA, dEdB, dv, X, Yd, N)
      
    a_medio = (al + au) / 2

    hl = h_linha(au, dEdA, dEdB, dv, X, Yd, N)

    itmax = np.int(np.ceil(np.log(au-al) - np.log(1e-5))/np.log(2)) #interações.

    #while (it < itmax) and ((au - al)<=1.0) and (np.abs(hl)>10.0):
    while (it < itmax):
      it += 1
      if hl > 0:
        au = a_medio
      elif hl < 0:
        al = a_medio
      elif hl == 0:
        return a_medio
      
      a_medio = (al + au) / 2
      hl = h_linha(au, dEdA, dEdB, dv, X, Yd, N)

    return a_medio

In [11]:
def calculo_saida(X, A, B, N):
  '''
    Calcula as predições para entradas usando argmax()
    X = Entrada da rede.
    A = Pesos da rede da camada do meio.
    b = Pesos da rede da camada de sáida.
    N = Número de instâncias do X.
    ________
    return: o vetor de saída com os valores das classes.
  '''
  Zin = np.dot(X, A.T)
  Z = 1/(1+np.exp(-Zin))

  Z = np.column_stack((Z,np.ones(N)))
  Yin = np.dot(Z, B.T)
  Y = 1/(1+np.exp(-Yin))

  return Y

In [12]:
def calculo_gradiente(X, Yd, A, B, N):
  Zin = np.dot(X, A.T) #(150,5)x(5xh)=(150,h)
  Z = 1/(1+np.exp(-Zin)) 

  Zb = np.column_stack((Z,np.ones(N))) #(150,h+1)
  Yin = np.dot(Zb, B.T) #(150, h+1)x(h+1,3) = (150,3) ns=3
  Y = 1/(1+np.exp(-Yin)) #(150,3)

  erro = Y -Yd 
  dg = (1-Y)*Y # (150,3) Derivada
  df = (1-Z)*Z # (150,h)
  

  dEdB = 1/N * np.dot((erro * dg).T, Zb) #(3,150)x(150,h+1)= (3,h+1)

  dEdZ = np.dot((erro* dg), B[:,:-1]) # retira o bias da miltiplicacao # (150,3)x(3,h)=(150, h)
  

  dEdA = 1/N * np.dot((dEdZ* df).T, X)  #(h,150)x(150,5)= (hx150)

  return dEdA, dEdB

In [13]:
def preditor_sigmoide(Y):
  '''
    Calcula as predições para entradas usando argmax()
    Y = relação com as saídas
    ________
    return: o vetor do resultado. 
  '''
  Y = Y.argmax(axis=1)
  return Y

In [14]:
def rna(X, Yd, h, taxa_aprendizado_fixa=None, verbose = False, epoca = 3000):
  N, ne = X.shape
  ns = Yd.shape[1]

  X = np.column_stack((X,np.ones(N)))
  seed(42)
  A = np.random.rand(h,(ne+1)) / 2  
  B = np.random.rand(ns,(h+1)) / 2  

  Y = calculo_saida(X, A, B, N)
  erro = Y - Yd


  MSE = 1/N * np.sum(erro*erro)
  
  ep = 0
  epmax = epoca
  E = 1e-2
  MSE_ant = 0
  MSE_atual = 0
  soma_MSE = 0
  
  a,b = [],[]# Utilizada no Gráfico.

  g =  np.concatenate([A.flatten(), B.flatten()])

  gradiente_norm = 1

  while (ep<epmax) and (soma_MSE < 20): #and (np.linalg.norm(g)>1e-7):
    ep += 1
    dEdA, dEdB = calculo_gradiente(X, Yd, A, B, N)

    if taxa_aprendizado_fixa is None:
      alfa = bissecao_rna(dEdA, dEdB, A, B, X, Yd, N)
    else:
      alfa = taxa_aprendizado_fixa

    A = A - alfa*dEdA
    B = B - alfa*dEdB

    #g =  np.concatenate([dEdA.flatten(), dEdB.flatten()])

    Y = calculo_saida(X, A, B, N)
    erro = Y - Yd
    
    
    MSE = 1/N * np.sum(erro*erro)

    if ((ep % 100)==0):
      print("Parâmetro :", h, "Epocas: ",ep, "MSE:", MSE, "ALFA:", alfa, 
          "Acurácia:",round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(Yd)),2))
    if verbose:
      a.append(ep)
      b.append(MSE)

    if MSE_ant == MSE: #Conta a quantidade de MSE iguais
      if MSE_ant == MSE_atual:
        soma_MSE +=1
      else:
        soma_MSE = 1
        MSE_atual = MSE
    else:  
      MSE_ant = MSE

  if verbose:
    fig = px.line(x= a, y=b, title='Gráfico - MLP()')
    fig.show()

  return A,B

## LBP

In [15]:
#Normalização Z-score
scaler = StandardScaler()
scaler.fit(image_lbp)
x_scaler = scaler.transform(image_lbp)

In [16]:
pca = PCA(n_components=50, random_state=42)
pca.fit(x_scaler)
X_pca = pca.transform(x_scaler)

In [17]:
soma = 0
for i in range(0, len(pca.explained_variance_ratio_)):
  soma += pca.explained_variance_ratio_[i]
  print(f'PC{ i+1}: {pca.explained_variance_ratio_[i]} - {round(soma*100,2)}%')

PC1: 0.07146507011961299 - 7.15%
PC2: 0.044204830641945046 - 11.57%
PC3: 0.03262730230382309 - 14.83%
PC4: 0.028766772332161746 - 17.71%
PC5: 0.0265049178850125 - 20.36%
PC6: 0.0242672737279358 - 22.78%
PC7: 0.022369431627340716 - 25.02%
PC8: 0.02024328636131167 - 27.04%
PC9: 0.018964531482152675 - 28.94%
PC10: 0.017541143178351544 - 30.7%
PC11: 0.016982223377488375 - 32.39%
PC12: 0.015653767805493428 - 33.96%
PC13: 0.014752780996445903 - 35.43%
PC14: 0.01397665642938275 - 36.83%
PC15: 0.012753728764836448 - 38.11%
PC16: 0.01123043664801963 - 39.23%
PC17: 0.010911527277293843 - 40.32%
PC18: 0.01007009607362514 - 41.33%
PC19: 0.009890287612781216 - 42.32%
PC20: 0.009024760898870925 - 43.22%
PC21: 0.008622142006023896 - 44.08%
PC22: 0.008260074919339692 - 44.91%
PC23: 0.008068954988169518 - 45.72%
PC24: 0.007671548910648472 - 46.48%
PC25: 0.007639729778018074 - 47.25%
PC26: 0.007020663583684242 - 47.95%
PC27: 0.0067956751876064 - 48.63%
PC28: 0.0065717502503128055 - 49.29%
PC29: 0.006347

In [18]:
#Normalização Z-score
scaler = StandardScaler()
scaler.fit(X_pca)
X_pca = scaler.transform(X_pca)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.30, random_state=42,stratify=target)

In [20]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((280, 50), (280, 40), (120, 50), (120, 40))

In [21]:
# Treino com melhor Parâmetro 
A, B = rna(x_train, y_train, 150, epoca=5000)

#Teste 
x_test_one = np.column_stack((x_test,np.ones(x_test.shape[0])))
Y = calculo_saida(x_test_one, A, B, x_test.shape[0])
print("Acurácia do Teste LBP:",round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(y_test)),2))

Parâmetro : 150 Epocas:  100 MSE: 0.8176389892691235 ALFA: 6.828370055471538 Acurácia: 0.21
Parâmetro : 150 Epocas:  200 MSE: 0.649675958422213 ALFA: 12.989290943603756 Acurácia: 0.38
Parâmetro : 150 Epocas:  300 MSE: 0.5558817735873987 ALFA: 17.656553250768674 Acurácia: 0.46
Parâmetro : 150 Epocas:  400 MSE: 0.5072452594311945 ALFA: 4034.2574278684287 Acurácia: 0.5
Parâmetro : 150 Epocas:  500 MSE: 0.4468448841338078 ALFA: 3232.9611007615263 Acurácia: 0.56
Parâmetro : 150 Epocas:  600 MSE: 0.3978277343534006 ALFA: 147.2856334867772 Acurácia: 0.6
Parâmetro : 150 Epocas:  700 MSE: 0.35072791142080734 ALFA: 5.569529105916908 Acurácia: 0.65
Parâmetro : 150 Epocas:  800 MSE: 0.3035832242234314 ALFA: 163493.43059234435 Acurácia: 0.7
Parâmetro : 150 Epocas:  900 MSE: 0.27508357611903694 ALFA: 2858.429679454463 Acurácia: 0.72
Parâmetro : 150 Epocas:  1000 MSE: 0.24653588850974134 ALFA: 43.3787892830688 Acurácia: 0.75
Parâmetro : 150 Epocas:  1100 MSE: 0.23928641611475712 ALFA: 76502.387901057

In [ ]:
%time
# Parametros
parametros = [100, 150] #, 200, 250]
data_atual = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
texto = ''
resultado_Classificador_LBP = []
resultadoFile = open('MLP_LBP_error.txt', 'w') # Armazena os resultados do treinamento
print('Execução em:', data_atual, file=resultadoFile)
print("====================================================================", file = resultadoFile)
print("====================================================================")
param_valores = []
kf = KFold(n_splits=2, shuffle=True, random_state=42)
kf.get_n_splits(x_train)
print(kf)

for p in parametros:
  print("+++++++++++++++")
  print("+++++++++++++++", file=resultadoFile)
  print("Tamanho da camada do meio da MLP: ", p)
  print("Tamanho da camada do meio da MLP: ", p, file=resultadoFile)
  print("+++++++++++++++")
  print("+++++++++++++++", file=resultadoFile)
  fold = 0
  valores = []
  for train_index, val_index in kf.split(x_train):
    fold +=1
    print("---------------")
    print("---------------", file=resultadoFile)
    print("Fold: ", fold)
    print("Fold: ", fold, file=resultadoFile)
    print("---------------")
    print("---------------", file=resultadoFile)
    X_train_kf, X_val_kf = x_train[train_index], x_train[val_index]
    y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
    
    seed(42)
      
    A, B = rna(X_train_kf, y_train_kf, p)

    X_val_kf_one = np.column_stack((X_val_kf,np.ones(X_val_kf.shape[0])))
    Y = calculo_saida(X_val_kf_one, A, B, X_val_kf.shape[0])

    print("Acurácia da Validação:",round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(y_val_kf)),2))
    print("Acurácia da Validação:",round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(y_val_kf)),2), file=resultadoFile)
    valores.append(round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(y_val_kf)),6)) 

  print("Parâmetro:", p, "Média Acurácia:", np.array(valores).mean())
  print("Parâmetro:", p, "Média Acurácia:", np.array(valores).mean(), file=resultadoFile)
  param_valores.append(np.array(valores).mean()) 
  
  
print("Médias da Acurácia dos Parâmetros: ", param_valores)
print("Médias da Acurácia dos Parâmetros: ", param_valores, file=resultadoFile)
melhor_parametro = parametros[np.array(param_valores).argmax()]
print("Melhor Parâmetro: ", melhor_parametro)
print("Melhor Parâmetro: ", melhor_parametro, file=resultadoFile)

resultadoFile.close()

# Armazena os paramentro de treinamento com o melhor deles.
configFile = open('MLP_LBP_config.txt', 'w') 
print('Execução em:', data_atual, file=configFile)
print("A rede foi treinada com os paramêtros:", parametros, file=configFile)
print("Os paramêtros são a quantidade de neurônios da camada do meio.", file=configFile)
print("Médias da Acurácia dos Parâmetros: ", param_valores, file=resultadoFile)
print("O melhor paramêtro:", melhor_parametro)
configFile.close()


# Treino com melhor Parâmetro 
A, B = rna(x_train, y_train, melhor_parametro, verbose=True)

#Teste 
X_test_one = np.column_stack((x_test,np.ones(x_test.shape[0])))
Y = calculo_saida(X_test_one, A, B, x_test.shape[0])
acuracia_teste = round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(y_test)),2)
print("Acurácia do Teste:",acuracia_teste)

resultado_Classificador_LBP = ['MLP', preditor_sigmoide(Y), acuracia_teste]

#Armazena o modelo treinado
modelFile = open('MLP_LBP_model.dat', 'w')
print("O melhor paramêtro para quantidade de neurônios:", melhor_parametro, file=modelFile)
print("A:", A, file=modelFile)
print("B:", A, file=modelFile)
modelFile.close()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs
KFold(n_splits=2, random_state=42, shuffle=True)
+++++++++++++++
Tamanho da camada do meio da MLP:  100
+++++++++++++++
---------------
Fold:  1
---------------
Parâmetro : 100 Epocas:  100 MSE: 0.8627743389341066 ALFA: 5.531628072712703 Acurácia: 0.19
Parâmetro : 100 Epocas:  200 MSE: 0.562839546519652 ALFA: 38.03612595046694 Acurácia: 0.46
Parâmetro : 100 Epocas:  300 MSE: 0.3716800443653164 ALFA: 30.615459738254643 Acurácia: 0.64
Parâmetro : 100 Epocas:  400 MSE: 0.3149016473976033 ALFA: 96.27440285614375 Acurácia: 0.69
Parâmetro : 100 Epocas:  500 MSE: 0.26433266916084047 ALFA: 6800.542439135218 Acurácia: 0.74
Parâmetro : 100 Epocas:  600 MSE: 0.25000016061667274 ALFA: 675398.7445423009 Acurácia: 0.75
Parâmetro : 100 Epocas:  700 MSE: 0.22142913751173685 ALFA: 627888.2599525747 Acurácia: 0.78
Parâmetro : 100 Epocas:  800 MSE: 0.2214285961618386 ALFA: 7367595.017043846 Acurácia: 0.78
Parâmetro : 100 Epocas:  900 MSE: 0.

## HOG

In [ ]:
#Normalização Z-score
scaler = StandardScaler()
scaler.fit(image_hog)
x_scaler = scaler.transform(image_hog)

In [ ]:
pca = PCA(n_components=50, random_state=42)
pca.fit(x_scaler)
X_pca = pca.transform(x_scaler)

In [ ]:
#Normalização Z-score
scaler = StandardScaler()
scaler.fit(X_pca)
X_pca = scaler.transform(X_pca)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.30, random_state=42,stratify=target)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((280, 50), (280, 40), (120, 50), (120, 40))

In [ ]:
# Treino com melhor Parâmetro 
A, B = rna(x_train, y_train, 100, epoca=3000)

#Teste 
x_test_one = np.column_stack((x_test,np.ones(x_test.shape[0])))
Y = calculo_saida(x_test_one, A, B, x_test.shape[0])
print("Acurácia do Teste HOG:",round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(y_test)),2))

Parâmetro : 100 Epocas:  100 MSE: 0.41813173073005006 ALFA: 4012.2793235841073 Acurácia: 0.6
Parâmetro : 100 Epocas:  200 MSE: 0.3685018329587846 ALFA: 1290.7074417416234 Acurácia: 0.64
Parâmetro : 100 Epocas:  300 MSE: 0.3289089344844218 ALFA: 693.9505213018442 Acurácia: 0.68
Parâmetro : 100 Epocas:  400 MSE: 0.321676709682063 ALFA: 12371.07382426484 Acurácia: 0.68
Parâmetro : 100 Epocas:  500 MSE: 0.30373488945737637 ALFA: 79.69385258524429 Acurácia: 0.7
Parâmetro : 100 Epocas:  600 MSE: 0.258827524063439 ALFA: 68.70510989787344 Acurácia: 0.74
Parâmetro : 100 Epocas:  700 MSE: 0.2571868863984063 ALFA: 505.86699900281064 Acurácia: 0.74
Parâmetro : 100 Epocas:  800 MSE: 0.22548540753824128 ALFA: 212.3499752072702 Acurácia: 0.78
Parâmetro : 100 Epocas:  900 MSE: 0.21428868686592792 ALFA: 13290.683603265185 Acurácia: 0.79
Parâmetro : 100 Epocas:  1000 MSE: 0.18072325398515193 ALFA: 16.268572480617536 Acurácia: 0.83
Parâmetro : 100 Epocas:  1100 MSE: 0.13571442192697386 ALFA: 7616237.1914

In [ ]:
preditor_sigmoide(Y)

array([ 9, 37, 23, 30, 22, 15, 22,  2, 19,  2, 34,  4,  3, 14,  1, 17, 13,
        5, 32,  7,  4,  3,  5,  0,  5, 18,  9, 31, 28, 17, 30, 32, 12,  6,
       24, 14, 36, 25, 21, 32,  7, 15, 11, 10, 35, 35, 26, 20, 34, 14, 30,
       28,  9,  4, 36, 20, 11, 25, 33, 24,  8, 29, 13, 25, 13, 38,  1, 29,
       21, 31, 19, 19, 39, 39,  6, 10, 16, 33, 31,  4,  8, 17,  7,  8,  0,
       27, 22, 19, 33, 29, 16, 28, 38, 14, 10, 16,  1, 11, 18, 37, 12, 27,
       27, 30, 38, 18, 31, 23,  0, 26, 26, 35, 24, 37,  6, 34, 31, 20, 15,
       37])

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly','sigmoid']} #'linea'
svc=svm.SVC(probability=True)
model=GridSearchCV(svc,param_grid)
model.fit(x_train,preditor_sigmoide(y_train))
model.best_params_

CPU times: user 15.6 s, sys: 42.2 ms, total: 15.6 s
Wall time: 15.5 s


{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

In [ ]:
y_train_pred = model.predict(x_train)
y_test_pred=model.predict(x_test)

print("Acurácia do Treinamento:", accuracy_score(preditor_sigmoide(y_train), y_train_pred))
print("Acurácia do Teste:", accuracy_score(preditor_sigmoide(y_test), y_test_pred))

Acurácia do Treinamento: 1.0
Acurácia do Teste: 0.9833333333333333


In [ ]:
y_test_pred

array([ 9, 21, 23, 30, 22, 15, 22,  2,  3,  2, 39,  4,  3, 14,  1, 17, 13,
       25, 32,  9,  4,  3,  5, 23,  5, 18,  9, 31, 28, 17, 30, 32, 12,  6,
       24, 14, 36, 36, 21, 32,  7, 15, 11, 10, 35, 35, 26, 20, 34, 14, 30,
       28,  9,  5, 36, 20, 11, 34, 33, 24,  8, 29, 13, 25, 13, 38,  1, 29,
       21, 31, 19, 19, 39, 39,  6, 10, 16, 33,  0,  2,  8, 17,  7,  8,  0,
       27, 22, 19, 33, 29, 16, 28, 38, 25, 10, 16,  1, 11, 18, 37, 12, 27,
       27, 30, 38, 18, 31, 23,  0, 26, 26, 35, 24, 37,  6, 34,  4, 20, 15,
       37])

In [ ]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
logit.fit(x_train, preditor_sigmoide(y_train))
y_train_pred = logit.predict(x_train)
y_test_pred= logit.predict(x_test)

print("Acurácia do Treinamento:", accuracy_score(preditor_sigmoide(y_train), y_train_pred))
print("Acurácia do Teste:", accuracy_score(preditor_sigmoide(y_test), y_test_pred))


Acurácia do Treinamento: 1.0
Acurácia do Teste: 0.9666666666666667


In [ ]:
y_test_pred

array([ 9, 21, 23, 30, 22, 15, 22,  2,  3,  2, 39,  4,  3, 14,  1, 17, 13,
       25, 32,  2,  4,  3,  5, 23,  5, 18,  9, 31, 28, 17, 30, 32, 12,  6,
       24, 14, 36, 36, 21, 32,  7, 15, 11, 10, 35, 35, 26, 20, 34, 14, 30,
       28,  9,  5, 36, 22, 11, 34, 33, 24,  8, 29, 13, 25, 13, 38,  1, 29,
       21, 31, 19, 19, 39, 39,  6, 10, 16, 33,  0,  2,  8, 17,  7,  8,  0,
       27, 22, 19, 33, 29, 16, 28, 38, 25, 10, 16,  1, 11, 18, 37, 12, 27,
       27, 30, 38, 18, 31, 23,  0, 26, 26, 35, 24, 37,  6, 34,  4, 22, 15,
       37])